# Feature extraction

# Импорты

In [1]:
%pip install pandas
%pip install pdfminer.six
%pip install scikit-learn
%pip install tqdm
%pip install transformers
%pip install pandarallel

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

from pdfminer.high_level import extract_pages, extract_text

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from tqdm import tqdm

## Загрузка данных для обучения

In [3]:
df_Train = pd.read_csv("books_train.csv")
df_Train, df_Test = train_test_split(df_Train, test_size=0.2, random_state=42)

# Функция для создания DF из PDF

In [4]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

def extract_df_from_pdf(book_name, folder_path):
    from pdfminer.high_level import extract_text
    
    def extract_text_from_page(book_name, folder_path, page):
        text = ""
        text += extract_text(f"{folder_path}/{book_name}", page_numbers=[page])
        return text

    len_book = len(list(extract_pages(f"{folder_path}/{book_name}")))
    df_book = pd.DataFrame(
        pd.Series(range(len_book)).parallel_apply(lambda page: [book_name, page, extract_text_from_page(book_name, folder_path, page)]).tolist(),
        columns=["book", "page_num", "text"]
    )
    return df_book


INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


## TF-IDF

In [5]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(df_Train["text"])
X_test = vectorizer.transform(df_Test["text"])

In [6]:
clf = RandomForestClassifier()
clf.fit(X_train, df_Train["label"])

RandomForestClassifier()

In [7]:
# Predict the test data
y_pred = clf.predict(X_test)

# Print the accuracy, precision, recall, and F1 score
print(f"Accuracy: {accuracy_score(df_Test['label'], y_pred)}")
print(f"Precision: {precision_score(df_Test['label'], y_pred, average='weighted')}")
print(f"Recall: {recall_score(df_Test['label'], y_pred, average='weighted')}")
print(f"F1 Score: {f1_score(df_Test['label'], y_pred, average='weighted')}")

Accuracy: 0.972972972972973
Precision: 0.9724796953150094
Recall: 0.972972972972973
F1 Score: 0.9724387658162493


In [8]:
# Check on random book
book_name = "Разработка_веб_приложения_GraphQL_с_React_Node_js_и_Neo4j.pdf"
folder_path = "./data/test"

book_df = extract_df_from_pdf(book_name, folder_path)

book_df["label"] = clf.predict(vectorizer.transform(book_df["text"]))
book_df[book_df["label"] == 1]

,book,page_num,text,label
6,Разработка_веб_приложения_GraphQL_с_React_Node...,6,Оглавление\n\nПредисловие от издательства .......,1
7,Разработка_веб_приложения_GraphQL_с_React_Node...,7,6  Оглавление\n\n1.7. Что мы будем строить ...,1
8,Разработка_веб_приложения_GraphQL_с_React_Node...,8,Оглавление  7\n\nГлава 4. Библиотека Neo4j ...,1
9,Разработка_веб_приложения_GraphQL_с_React_Node...,9,8  Оглавление\n\nГлава 6. Клиент GraphQL .....,1
10,Разработка_веб_приложения_GraphQL_с_React_Node...,10,Оглавление  9\n\n8.2.3. Выгрузка данных в N...,1
256,Разработка_веб_приложения_GraphQL_с_React_Node...,256,узел﻿и﻿отношение﻿﻿70\nузел﻿и﻿свойство﻿﻿69\nдан...,1
259,Разработка_веб_приложения_GraphQL_с_React_Node...,259,258  Предметный﻿указатель\n\nвычисляемые﻿по...,1
260,Разработка_веб_приложения_GraphQL_с_React_Node...,260,MERGE﻿команда﻿﻿76\nmoviesByTitle﻿функция﻿разре...,1


### Попытка сделать то же, но с предобработкой

In [9]:
def preprocess_text(text):
    text = text.lower()
    text = text.replace("\n", " ")
    text = text.replace("\t", " ")
    return text

vectorizer_ = TfidfVectorizer()
X_train_ = vectorizer_.fit_transform(df_Train["text"].apply(preprocess_text))
X_test_ = vectorizer_.transform(df_Test["text"].apply(preprocess_text))

In [10]:
clf_ = RandomForestClassifier()
clf_.fit(X_train_, df_Train["label"])

RandomForestClassifier()

In [11]:
# Predict the test data
y_pred_ = clf_.predict(X_test_)
print(f"Accuracy: {accuracy_score(df_Test['label'], y_pred_)}")
print(f"Precision: {precision_score(df_Test['label'], y_pred_)}")
print(f"Recall: {recall_score(df_Test['label'], y_pred_)}")
print(f"F1 Score: {f1_score(df_Test['label'], y_pred_)}")

Accuracy: 0.9768339768339769
Precision: 0.9666666666666667
Recall: 0.8529411764705882
F1 Score: 0.90625


In [12]:
book_df["label"] = clf_.predict(vectorizer_.transform(book_df["text"]))
book_df[book_df["label"] == 1]

,book,page_num,text,label
6,Разработка_веб_приложения_GraphQL_с_React_Node...,6,Оглавление\n\nПредисловие от издательства .......,1
8,Разработка_веб_приложения_GraphQL_с_React_Node...,8,Оглавление  7\n\nГлава 4. Библиотека Neo4j ...,1
9,Разработка_веб_приложения_GraphQL_с_React_Node...,9,8  Оглавление\n\nГлава 6. Клиент GraphQL .....,1
10,Разработка_веб_приложения_GraphQL_с_React_Node...,10,Оглавление  9\n\n8.2.3. Выгрузка данных в N...,1
256,Разработка_веб_приложения_GraphQL_с_React_Node...,256,узел﻿и﻿отношение﻿﻿70\nузел﻿и﻿свойство﻿﻿69\nдан...,1
257,Разработка_веб_приложения_GraphQL_с_React_Node...,257,256  Предметный﻿указатель\n\nупорядочение\n...,1
259,Разработка_веб_приложения_GraphQL_с_React_Node...,259,258  Предметный﻿указатель\n\nвычисляемые﻿по...,1
260,Разработка_веб_приложения_GraphQL_с_React_Node...,260,MERGE﻿команда﻿﻿76\nmoviesByTitle﻿функция﻿разре...,1
261,Разработка_веб_приложения_GraphQL_с_React_Node...,261,260  Предметный﻿указатель\n\noffset﻿аргумен...,1


## Word2Vec

## ruBERT

In [13]:
from transformers import BertTokenizer, BertModel
import torch

In [14]:
# Загрузка предварительно обученного токенайзера и модели ruBERT
tokenizer = BertTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
model = BertModel.from_pretrained("DeepPavlov/rubert-base-cased")

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

# Move the model to the specified device
model = model.to(device)

cuda


In [16]:
# Function to get BERT embeddings using the specified device
def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
    
    # Move inputs to the same device as the model
    inputs = {key: tensor.to(device) for key, tensor in inputs.items()}
    
    # Get embeddings using the model
    outputs = model(**inputs)
    
    # Get the last hidden state from the model output
    last_hidden_states = outputs.last_hidden_state
    
    # Move the last hidden state to CPU if it was calculated on GPU
    last_hidden_states = last_hidden_states.cpu().detach()
    
    # Average over tokens to get the final embedding
    text_embedding = torch.mean(last_hidden_states, dim=1)
    
    # Move the tensor to the GPU if available
    text_embedding = text_embedding.to(device)
    
    return text_embedding

In [17]:
# Convert the data to CUDA tensors if available
X_train_bert = [get_bert_embeddings(text) for text in tqdm(df_Train['text'])]
X_test_bert = [get_bert_embeddings(text) for text in tqdm(df_Test['text'])]

100%|████████████████████████████████████████████████████████████████████████████████| 259/259 [01:13<00:00,  3.53it/s]


In [18]:
# Convert the list of tensors to a single tensor
X_train_bert = torch.cat(X_train_bert, dim=0)
X_test_bert = torch.cat(X_test_bert, dim=0)

# Convert the labels to a tensor
y_train_bert = torch.tensor(df_Train['label'].values)
y_test_bert = torch.tensor(df_Test['label'].values)


In [19]:
# Train the model
clf_bert = RandomForestClassifier()
clf_bert.fit(X_train_bert.cpu(), y_train_bert)

RandomForestClassifier()

In [20]:
# Predict the test data
y_pred_bert = clf_bert.predict(X_test_bert.cpu())

# Print the accuracy, precision, recall, and F1 score
print(f"Accuracy: {accuracy_score(y_test_bert, y_pred_bert)}")
print(f"Precision: {precision_score(y_test_bert, y_pred_bert, average='weighted')}")
print(f"Recall: {recall_score(y_test_bert, y_pred_bert, average='weighted')}")
print(f"F1 Score: {f1_score(y_test_bert, y_pred_bert, average='weighted')}")

Accuracy: 0.9845559845559846
Precision: 0.9848257490178888
Recall: 0.9845559845559846
F1 Score: 0.9841413943836851


In [64]:
# Check on random book
book_name = "Объекты_Стильное_ООП.pdf"
folder_path = "./data/readable"

book_df = extract_df_from_pdf(book_name, folder_path)

In [65]:
# Extract BERT embeddings for the text
embeddings = [get_bert_embeddings(text) for text in tqdm(book_df["text"])]

100%|████████████████████████████████████████████████████████████████████████████████| 304/304 [01:08<00:00,  4.43it/s]


In [66]:
# Convert the list of tensors to a single tensor
X_book_bert = torch.cat(embeddings, dim=0)

In [67]:
# Predict the labels
book_df["label"] = clf_bert.predict(X_book_bert.cpu())
book_df[book_df["label"] == 1]

,book,page_num,text,label
4,Объекты_Стильное_ООП.pdf,4,Краткое содержание\n\nПредисловие................,1
5,Объекты_Стильное_ООП.pdf,5,Оглавление\n\nПредисловие........................,1
6,Объекты_Стильное_ООП.pdf,6,Оглавление \n\n 7\n\n1.9..Оператор.возврата.и...,1
7,Объекты_Стильное_ООП.pdf,7,8 \n\n Оглавление\n\n3.2..Запрашивайте.только...,1
8,Объекты_Стильное_ООП.pdf,8,Оглавление \n\n 9\n\n4.2..Объекты-значения:.з...,1
9,Объекты_Стильное_ООП.pdf,9,10 \n\n Оглавление\n\n5.2.2..Именование.недоп...,1
10,Объекты_Стильное_ООП.pdf,10,Оглавление \n\n 11\n\n7.6..Задавайте.абстрак...,1
11,Объекты_Стильное_ООП.pdf,11,12 \n\n Оглавление\n\n10.3..Репозитории.модел...,1


In [68]:
# Concat text of each page of content table
content_table = book_df[book_df["label"] == 1]["text"].str.cat(sep=" ")

In [69]:
print(content_table)

Краткое содержание

Предисловие...............................................................................................................................14

Вступительное.слово...............................................................................................................16

Благодарности............................................................................................................................17

О.книге..........................................................................................................................................18

Об.авторе......................................................................................................................................21

Иллюстрации.на.обложке......................................................................................................22

От.издательства.........................................................................................................................23


In [70]:
import re
# Create function to create ierarchy of content table (split by \n, but before - remove all double \n)
def split_content_table(content_table):
    content_table = content_table.replace("\n\n", "\n")
    content_table = content_table.split("\n")
    return content_table
        
def parse_table_of_contents(content):
    lines = content.split('\n')
    toc = []

    current_part = None
    current_chapter = None

    for line in lines:
        if 'Глава' in line:
            current_chapter = line.strip()
            if current_part is None:
                toc.append({'title': current_chapter, 'subsections': []})
            else:
                toc[-1]['subsections'].append({'title': current_chapter, 'subsections': []})
        elif '.' in line:
            depth = line.count('.')
            title = line.strip()
            if current_chapter is None:
                toc.append({'title': title, 'subsections': []})
            else:
                toc[-1]['subsections'][-1]['subsections'].append({'title': title, 'subsections': []})

    return toc


content_table_splitted = split_content_table(content_table)
parsed_content_table = parse_table_of_contents(content_table)

In [71]:
# Function to print parsed content table (for each level of hierarchy print with tabulation)
def print_toc(parsed_content_table, level=0):
    for item in parsed_content_table:
        print("\t" * level + item["title"])
        print_toc(item["subsections"], level + 1)
        
print_toc(parsed_content_table)

Предисловие...............................................................................................................................14
Вступительное.слово...............................................................................................................16
Благодарности............................................................................................................................17
О.книге..........................................................................................................................................18
Об.авторе......................................................................................................................................21
Иллюстрации.на.обложке......................................................................................................22
От.издательства.........................................................................................................................23
Глава 1. Программирование.

In [72]:
print(parsed_content_table)

[{'title': 'Предисловие...............................................................................................................................14', 'subsections': []}, {'title': 'Вступительное.слово...............................................................................................................16', 'subsections': []}, {'title': 'Благодарности............................................................................................................................17', 'subsections': []}, {'title': 'О.книге..........................................................................................................................................18', 'subsections': []}, {'title': 'Об.авторе......................................................................................................................................21', 'subsections': []}, {'title': 'Иллюстрации.на.обложке.............................................................................................